In [1]:
!nvidia-smi

Thu Jun 16 08:09:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%pip install -q transformers[sentencepiece] datasets

     |████████████████████████████████| 4.2 MB 10.8 MB/s 
     |████████████████████████████████| 362 kB 12.0 MB/s 
     |████████████████████████████████| 212 kB 15.3 MB/s 
     |████████████████████████████████| 1.1 MB 64.2 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 140 kB 77.2 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |████████████████████████████████| 127 kB 76.2 MB/s 
     |████████████████████████████████| 271 kB 70.4 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 76.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.6 MB/s 
     |████████████████████████████████| 1.2 MB 66.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which 

### In this notebook we experiment by finetuning a bert-for-patents model on competition data by adding patent section as special token to the tokenizer vocab.

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)

In [2]:
import numpy as np
import pandas as pd
import torch
import os
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
from torch.utils.data import DataLoader
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch.multiprocessing as mp
from transformers import logging
import warnings

logging.set_verbosity_error()
logging.set_verbosity_warning()
warnings.filterwarnings('ignore')


In [3]:
class TrainingArgs:
    weight_decay = 0.01
    learning_rate = 1e-5
    warmup_ratio = 0.1
    gradient_accumulation_steps = 2
    fp16 = True
    lr_scheduler_type = "linear"
    # Number of checkpoints to save for each model
    save_total_limit = 1
    #  Whether or not to load the best model found during training at the end of training.
    load_best_model_at_end=True
    # Use in conjunction with `load_best_model_at_end` to specify the metric to use to compare two different
    # models. Must be the name of a metric returned by the evaluation with or without the prefix `"eval_"`. Will
    # default to `"loss"` if unspecified and `load_best_model_at_end=True` (to use the evaluation loss).
    # If you set this value, `greater_is_better` will default to `True`. Don't forget to set it to `False` if
    # your metric is better when lower.
    metric_for_best_model="loss"
    adam_epsilon=1e-6
    #warmup_steps=1000
    log_level="warning"

class Config:
    MODEL_NAME = "deberta-v3-large"
    DATA_PATH = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/data/"
    VAL_PREDS_PATH = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/preds/"
    # location where trained model weights are saved
    OUT_DIR = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/deberta-v3-large/"
    RUNTIME = "COLAB"
    RANDOM_STATE = 42
    BATCH_SIZE = 24
    EVAL_BATCH_SIZE = 64
    NUM_LABELS = 1
    NUM_FOLDS = 5
    RUN_ALL_FOLDS = True
    NUM_EPOCHS = 7
    NUM_WORKERS = mp.cpu_count()
    TRANSFORMER_CHECKPOINT = "microsoft/deberta-v3-large"
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SUBSET_ROWS_FRAC = 0.05
    TRAIN_ON_SUBSET = False
    RANDOM_SEED = 42

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
def empty_gdrive_trash():
    deleted_file_name = []
    for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
        file_name = a_file['title']
        deleted_file_name.append(file_name)
        # delete the file permanently.
        a_file.Delete()
    print("The below files were cleared from trash")
    print(deleted_file_name)

In [5]:
empty_gdrive_trash()

The below files were cleared from trash
[]


In [6]:
df_train = pd.read_csv(Config.DATA_PATH + "train.csv")
df_test = pd.read_csv(Config.DATA_PATH + "test.csv")
df_titles = pd.read_csv(Config.DATA_PATH + "titles.csv")

In [7]:
df_train["section"] = df_train.context.str[0]

In [8]:
from sklearn.preprocessing import LabelEncoder

anchor_encoder = LabelEncoder()
df_train["anchor_map"] = anchor_encoder.fit_transform(df_train["anchor"])
# Score is not really a continuous value here as there are just five distinct values. But since it is float it needs to be converted
# to categorical value before we can perform stratified split on score
df_train["score_map"] = df_train["score"].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})

In [9]:
from sklearn import model_selection

def strat_group_kfold_dataframe(df, target_col_name, group_col_name, num_folds=Config.NUM_FOLDS):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df = df.sample(frac=1, random_state=Config.RANDOM_SEED).reset_index(drop=True)
    # get the target data
    y = df[target_col_name].values    
    groups = df[group_col_name].values
    # stratify data using anchor as group and score as target
    skf = model_selection.StratifiedGroupKFold(n_splits=num_folds, shuffle=True, random_state=Config.RANDOM_SEED)
    for fold, (train_index, val_index) in enumerate(skf.split(X=df, y=y, groups=groups)):
        df.loc[val_index, "kfold"] = fold        
    return df     

In [10]:
def strat_kfold_dataframe(df, target_col_name, num_folds=5):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df.sample(frac=1, random_state=Config.RANDOM_STATE).reset_index(drop=True)
    y = df[target_col_name].values
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=Config.RANDOM_STATE)
    # stratification is done on the basis of y labels, a placeholder for X is sufficient
    for fold, (train_idx, val_idx) in enumerate(skf.split(X=df, y=y)):
        df.loc[val_idx, "kfold"] = fold
    return df

In [11]:
if Config.TRAIN_ON_SUBSET:
    print(f"Selecting {Config.SUBSET_ROWS_FRAC * 100}% training data")
    df_train = df_train.sample(frac=Config.SUBSET_ROWS_FRAC, random_state=Config.RANDOM_SEED).reset_index(drop=True)

# Since the target column (score) is continuous, we need to create bins out of the target column
# df_train.loc[:, "bins"] = pd.cut(df_train.score, bins=5, labels=[0,1,2,3,4])
# df_train = strat_kfold_dataframe(df_train, target_col_name="bins", num_folds=Config.NUM_FOLDS)

# Now do a stratified group k fold on the bins column (which is a categorical column) and anchor as groups
df_train = strat_group_kfold_dataframe(df_train, target_col_name="score_map", group_col_name="anchor_map", num_folds=Config.NUM_FOLDS)            
# drop the bin column
# df_train = df_train.drop(["bins"], axis=1)
# df_train = df_train.drop(["anchor_map", "score_map"], axis=1)

In [12]:
# Let us check if the stratification has been done correctly
# The mean of score column should be similar across folds 
fold_score_mean = []
fold_anchor_maps = []
for fold in range(Config.NUM_FOLDS):
    df_train_fold = df_train[df_train.kfold == fold]
    fold_score_mean.append(np.mean(df_train_fold.score.values))
    fold_anchor_maps.append(set(df_train_fold.anchor_map.unique()))
fold_score_mean

[0.36078538314698666,
 0.3523398128149748,
 0.3631549993123367,
 0.36788755237194215,
 0.3654970760233918]

In [13]:
# check each of the folds has no common anchor value
def check_disjoint(start, fold_anchor_maps):
    for i in range(start, 4):
        for j in range(i+1, 5):
            if fold_anchor_maps[i].isdisjoint(fold_anchor_maps[j]):
                print(f"anchor map for fold {i} and {j} are disjoint")

check_disjoint(0, fold_anchor_maps)                

anchor map for fold 0 and 1 are disjoint
anchor map for fold 0 and 2 are disjoint
anchor map for fold 0 and 3 are disjoint
anchor map for fold 0 and 4 are disjoint
anchor map for fold 1 and 2 are disjoint
anchor map for fold 1 and 3 are disjoint
anchor map for fold 1 and 4 are disjoint
anchor map for fold 2 and 3 are disjoint
anchor map for fold 2 and 4 are disjoint
anchor map for fold 3 and 4 are disjoint


In [14]:
df_titles.head()

,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


In [15]:
df_train = pd.merge(
    left = df_train,
    right = df_titles[["code", "title"]],
    how = "inner",
    left_on = "context",
    right_on = "code"
)

In [16]:
df_train.head()

,id,anchor,target,context,score,section,anchor_map,score_map,kfold,code,title
0,ed1c4e525eb105fe,transmit alarm,display indicator,G08,0.00,G,675,0,1,G08,SIGNALLING
1,c261266d1ba87355,running tally,stationary items,G08,0.00,G,584,0,3,G08,SIGNALLING
2,499fd13434840bd3,main lane,entry lane,G08,0.25,G,389,1,3,G08,SIGNALLING
3,5889ce7d8b2da774,intruder detection,electronic information processing,G08,0.50,G,355,2,1,G08,SIGNALLING
4,9430304cd83c9637,calling card,numbers,G08,0.25,G,87,1,4,G08,SIGNALLING


In [17]:
import torch.nn as nn
from typing import Optional, Union, Tuple
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import DebertaV2Model, DebertaV2PreTrainedModel
from transformers.models.deberta.modeling_deberta import ContextPooler, StableDropout

def get_loss(problem_type, num_labels, labels, logits):
    loss = None
    if labels is not None:
        if problem_type is None:
            if num_labels == 1:
                # regression task
                loss_fn = nn.MSELoss()
                logits = logits.view(-1).to(labels.dtype)
                loss = loss_fn(logits, labels.view(-1))
            elif labels.dim() == 1 or labels.size(-1) == 1:
                label_index = (labels >= 0).nonzero()
                labels = labels.long()
                if label_index.size(0) > 0:
                    labeled_logits = torch.gather(
                        logits, 0, label_index.expand(label_index.size(0), logits.size(1))
                    )
                    labels = torch.gather(labels, 0, label_index.view(-1))
                    loss_fct = nn.CrossEntropyLoss()
                    loss = loss_fct(labeled_logits.view(-1, num_labels).float(), labels.view(-1))
                else:
                    loss = torch.tensor(0).to(logits)
            else:
                log_softmax = nn.LogSoftmax(-1)
                loss = -((log_softmax(logits) * labels).sum(-1)).mean()
        elif problem_type == "regression":
            loss_fct = nn.MSELoss()
            if num_labels == 1:
                loss = loss_fct(logits.squeeze(), labels.squeeze())
            else:
                loss = loss_fct(logits, labels)
        elif problem_type == "single_label_classification":
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))
        elif problem_type == "multi_label_classification":
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)
    return loss

In [18]:
class DebertaV2ForSeqClfMeanPooling(DebertaV2PreTrainedModel):    
    def __init__(self, config):
        super().__init__(config)

        num_labels = getattr(config, "num_labels", 2)
        self.num_labels = num_labels

        self.deberta = DebertaV2Model(config)
        self.pooler = ContextPooler(config)
        output_dim = self.pooler.output_dim

        self.classifier = nn.Linear(output_dim, num_labels)
        drop_out = getattr(config, "cls_dropout", None)
        drop_out = self.config.hidden_dropout_prob if drop_out is None else drop_out
        self.dropout = StableDropout(drop_out)

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.deberta.get_input_embeddings()

    def set_input_embeddings(self, new_embeddings):
        self.deberta.set_input_embeddings(new_embeddings)
    
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.deberta(
            input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # hidden state from the last layer [batch_size, seq_len, hidden_size]
        last_hidden_state = outputs[0]
        # copy the 2d attention mask [batch_size, seq_len] hidden_size times in the third dimension (hidden state)
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        # [batch_size, seq_len, hidden_size ]
        # Each hidden state is a tensor of dimension [batch_size, seq_len] and we have hidden_size number of such hidden state
        # Of these seq_len columns in each hidden state only those need to be taken into account for which attention_mask = 1.  
        # Doing an element wise multiplication of the 2d attention mask [batch_size, seq_len] with the corresponding 2d hidden state 
        # [batch_size, seq_len] gives hidden state with only the non-padded columns. Sum this hidden state along dimension 1 
        # ( the dimension of sequence length) to get the sum_embeddings [batch_size, hidden_size] 
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        # The sum mask is a value between 0 to 256 signifying the number of unpadded columns for that hidden state
        sum_mask = input_mask_expanded.sum(1)
        # [batch_size, hidden_size]
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        # element wise division 
        mean_embeddings = sum_embeddings / sum_mask
        # [batch_size, hidden_size]
        logits = self.classifier(mean_embeddings)
        loss = get_loss(self.config.problem_type, self.num_labels, labels, logits)
        if not return_dict:
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions
        )


In [19]:
class DebertaV2ForSeqClfMaxPooling(DebertaV2PreTrainedModel):    
    def __init__(self, config):
        super().__init__(config)

        num_labels = getattr(config, "num_labels", 2)
        self.num_labels = num_labels

        self.deberta = DebertaV2Model(config)
        self.pooler = ContextPooler(config)
        output_dim = self.pooler.output_dim

        self.classifier = nn.Linear(output_dim, num_labels)
        drop_out = getattr(config, "cls_dropout", None)
        drop_out = self.config.hidden_dropout_prob if drop_out is None else drop_out
        self.dropout = StableDropout(drop_out)

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.deberta.get_input_embeddings()

    def set_input_embeddings(self, new_embeddings):
        self.deberta.set_input_embeddings(new_embeddings)
    
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.deberta(
            input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # hidden state from the last layer [batch_size, seq_len, hidden_size]
        last_hidden_state = outputs[0]
        # copy the 2d attention mask [batch_size, seq_len] hidden_size times in the third dimension (hidden state)
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        # [batch_size, seq_len, hidden_size ]        
        # Set padding tokens to large negative value
        last_hidden_state[input_mask_expanded == 0] = -1e9  
        # Get the max value along dimension 1 (seq_len dimension) of last_hidden_state. The result will be [batch_size, hidden_size] tensor
        # The first output torch.max returns the max values along a dimension, the second output is the index of max value
        max_embeddings = torch.max(last_hidden_state, 1)[0]        
        # [batch_size, hidden_size]
        logits = self.classifier(max_embeddings)
        loss = get_loss(self.config.problem_type, self.num_labels, labels, logits)
        if not return_dict:
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions
        )


In [20]:
class DebertaV2ForSeqClfMeanMaxPooling(DebertaV2PreTrainedModel):    
    def __init__(self, config):
        super().__init__(config)

        num_labels = getattr(config, "num_labels", 2)
        self.num_labels = num_labels

        self.deberta = DebertaV2Model(config)
        self.pooler = ContextPooler(config)
        output_dim = self.pooler.output_dim
        
        self.classifier = nn.Linear(2*output_dim, num_labels)
        drop_out = getattr(config, "cls_dropout", None)
        drop_out = self.config.hidden_dropout_prob if drop_out is None else drop_out
        self.dropout = StableDropout(drop_out)

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.deberta.get_input_embeddings()

    def set_input_embeddings(self, new_embeddings):
        self.deberta.set_input_embeddings(new_embeddings)
    
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.deberta(
            input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        last_hidden_state = outputs[0]        
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)        
        sum_mask = input_mask_expanded.sum(1)        
        sum_mask = torch.clamp(sum_mask, min=1e-9)        
        mean_pooling_embeddings = sum_embeddings / sum_mask        

        last_hidden_state[input_mask_expanded == 0] = -1e9          
        max_pooling_embeddings = torch.max(last_hidden_state, 1)[0]

        mean_max_embeddings = torch.cat((mean_pooling_embeddings, max_pooling_embeddings), 1)
        # [batch_size, 2*hidden_size]
        logits = self.classifier(mean_max_embeddings)
        loss = get_loss(self.config.problem_type, self.num_labels, labels, logits)
        if not return_dict:
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions
        )


In [21]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(Config.TRANSFORMER_CHECKPOINT)
# DataCollatorWithPadding pads each batch to the longest sequence length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# model = AutoModelForSequenceClassification.from_pretrained(Config.TRANSFORMER_CHECKPOINT, num_labels=Config.NUM_LABELS)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
df_train['sectok'] = '[' + df_train.section + ']'
sectoks = list(df_train.sectok.unique())
sep = '[s]'
sectoks.append(sep)
print(f"Additional special tokens: {sectoks}")
tokenizer.add_special_tokens({'additional_special_tokens': sectoks})

Additional special tokens: ['[G]', '[B]', '[H]', '[A]', '[F]', '[C]', '[D]', '[E]', '[s]']


9

In [23]:
#sep = tokenizer.sep_token
df_train["inputs"] = df_train.sectok + sep + df_train.anchor + sep + df_train.target + sep + df_train.title 
df_train.head()

,id,anchor,target,context,score,section,anchor_map,score_map,kfold,code,title,sectok,inputs
0,ed1c4e525eb105fe,transmit alarm,display indicator,G08,0.00,G,675,0,1,G08,SIGNALLING,[G],[G][s]transmit alarm[s]display indicator[s]SIG...
1,c261266d1ba87355,running tally,stationary items,G08,0.00,G,584,0,3,G08,SIGNALLING,[G],[G][s]running tally[s]stationary items[s]SIGNA...
2,499fd13434840bd3,main lane,entry lane,G08,0.25,G,389,1,3,G08,SIGNALLING,[G],[G][s]main lane[s]entry lane[s]SIGNALLING
3,5889ce7d8b2da774,intruder detection,electronic information processing,G08,0.50,G,355,2,1,G08,SIGNALLING,[G],[G][s]intruder detection[s]electronic informat...
4,9430304cd83c9637,calling card,numbers,G08,0.25,G,87,1,4,G08,SIGNALLING,[G],[G][s]calling card[s]numbers[s]SIGNALLING


In [24]:
def tokenize_text(tokenizer, with_labels, row):
    encoding = tokenizer(
        text = row["inputs"],
        padding = False,
        truncation = True
    )
    if with_labels:
        encoding["labels"] = row["score"]
    return encoding

In [25]:
from functools import partial

preprocess_train_data = partial(tokenize_text, tokenizer, True)  
preprocess_test_data = partial(tokenize_text, tokenizer, False)  

In [26]:
def get_fold_dls(fold, df):
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    ds_train_raw = Dataset.from_pandas(train_df)
    ds_valid_raw = Dataset.from_pandas(valid_df)
    raw_ds_col_names = ds_train_raw.column_names    
    ds_train = ds_train_raw.map(preprocess_train_data, batched=True, batch_size=1000, remove_columns=raw_ds_col_names)
    ds_valid = ds_valid_raw.map(preprocess_train_data, batched=True, batch_size=1000, remove_columns=raw_ds_col_names)    
    return train_df, valid_df, ds_train, ds_valid

In [27]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [28]:
def get_oof_preds(trainer, ds_val, df_val_fold):
    oof_outputs = trainer.predict(ds_val)
    oof_predictions = oof_outputs.predictions.reshape(-1)
    df_val_fold["val_preds"] = oof_predictions
    return df_val_fold

In [29]:
def get_training_args(fold_str):
    training_args = TrainingArguments(
        output_dir=Config.OUT_DIR + fold_str,
        evaluation_strategy="epoch",
        save_strategy='epoch',        
        num_train_epochs=Config.NUM_EPOCHS,
        per_device_train_batch_size=Config.BATCH_SIZE,
        per_device_eval_batch_size=Config.EVAL_BATCH_SIZE,
        warmup_ratio=TrainingArgs.warmup_ratio,
        weight_decay=TrainingArgs.weight_decay,
        learning_rate=TrainingArgs.learning_rate,    
        gradient_accumulation_steps=TrainingArgs.gradient_accumulation_steps,
        fp16=TrainingArgs.fp16,
        lr_scheduler_type=TrainingArgs.lr_scheduler_type,
        save_total_limit=TrainingArgs.save_total_limit,
        load_best_model_at_end=TrainingArgs.load_best_model_at_end,
        metric_for_best_model=TrainingArgs.metric_for_best_model,
        adam_epsilon=TrainingArgs.adam_epsilon,
        #warmup_steps=TrainingArgs.warmup_steps,
        log_level=TrainingArgs.log_level
    )
    return training_args

In [30]:
import gc

df_val_preds = pd.DataFrame()
tok_vocab = tokenizer.get_vocab()
for fold in range(Config.NUM_FOLDS):
    fold_str = f"fold{fold}"
    print(f"Running training for {fold_str}")
    df_train_fold, df_val_fold, ds_train, ds_val = get_fold_dls(fold, df_train)
    training_args = get_training_args(fold_str)
    model = AutoModelForSequenceClassification.from_pretrained(Config.TRANSFORMER_CHECKPOINT, num_labels=Config.NUM_LABELS)
    print(f"len(tokenizer_vocab) = {len(tok_vocab)}")
    model.resize_token_embeddings(len(tok_vocab))    
    trainer = Trainer(
        model=model,                         # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=ds_train,              # training dataset
        eval_dataset=ds_val,                 # evaluation dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
        data_collator=data_collator,
        tokenizer=tokenizer
    )
    trainer.train()
    trainer.save_model(Config.OUT_DIR + fold_str)
    df_val_fold = get_oof_preds(trainer, ds_val, df_val_fold) 
    display(df_val_fold.head())
    df_val_preds = pd.concat([df_val_preds, df_val_fold], axis=0)
    # export the oof predictions to csv for later use in stacking
    if Config.RUNTIME != "KAGGLE":
        df_val_fold.to_csv(Config.VAL_PREDS_PATH + f"df_train_oof_preds_{Config.MODEL_NAME}_{fold_str}.csv")
    else:
        df_val_preds.to_csv("/kaggle/working/df_train_oof_preds.csv")
    print(f"Saved OOF predictions for fold {fold}")    
    del model, trainer
    gc.collect()
    torch.cuda.empty_cache()
    # Empty the trash to clear gdrive disk space
    empty_gdrive_trash()
    if not Config.RUN_ALL_FOLDS:
        break

if Config.RUN_ALL_FOLDS:
    df_val_preds.to_csv(Config.VAL_PREDS_PATH + f"df_train_oof_preds_{Config.MODEL_NAME}.csv")

Parameter 'function'=<function Dataset.map.<locals>.decorate.<locals>.decorated at 0x7f3658845ef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Running training for fold0


  0%|          | 0/30 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/8 [00:00<?, ?ba/s]

...

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

len(tokenizer_vocab) = 128010


Epoch,Training Loss,Validation Loss,Pearson
0,0.066100,0.023690,0.822290
1,0.024100,0.022630,0.835162
2,0.017700,0.024026,0.832985
3,0.014800,0.021829,0.838987
4,0.010600,0.021865,0.838554
5,0.009200,0.022881,0.837283
6,0.008700,0.022431,0.837260


,id,anchor,target,context,score,section,anchor_map,score_map,kfold,code,title,sectok,inputs,val_preds
0,9430304cd83c9637,calling card,numbers,G08,0.25,G,87,1,4,G08,SIGNALLING,[G],[G][s]calling card[s]numbers[s]SIGNALLING,0.198486
1,a6b3dc244eba23cf,central communication unit,transmitter,G08,0.25,G,98,1,4,G08,SIGNALLING,[G],[G][s]central communication unit[s]transmitter...,0.392090
2,629b0fa8ce54898a,central communication unit,communication control device,G08,0.50,G,98,2,4,G08,SIGNALLING,[G],[G][s]central communication unit[s]communicati...,0.547852
3,b9121277a75fdf01,central communication unit,master lock,G08,0.00,G,98,0,4,G08,SIGNALLING,[G],[G][s]central communication unit[s]master lock...,0.116577
4,2033ed2dd902255b,calling card,exemplary telecommunications apparatus,G08,0.50,G,87,2,4,G08,SIGNALLING,[G],[G][s]calling card[s]exemplary telecommunicati...,0.397705


Saved OOF predictions for fold 4
The below files were cleared from trash
['rng_state.pth', 'trainer_state.json', 'scaler.pt', 'scheduler.pt', 'training_args.bin', 'tokenizer.json', 'spm.model', 'added_tokens.json', 'special_tokens_map.json', 'tokenizer_config.json', 'config.json', 'rng_state.pth', 'trainer_state.json', 'scaler.pt', 'scheduler.pt', 'optimizer.pt', 'training_args.bin', 'tokenizer.json', 'spm.model', 'added_tokens.json', 'special_tokens_map.json', 'tokenizer_config.json', 'pytorch_model.bin', 'config.json', 'checkpoint-603']


In [31]:
# Calculate the CV score
predictions = df_val_preds['val_preds'].values
labels = df_val_preds['score'].values
eval_preds = predictions, labels
cv_metric_dict = compute_metrics(eval_preds)
print(f"CV score = {cv_metric_dict}")

CV score = {'pearson': 0.8271040281123369}


In [32]:
df_val_preds["score_pred_diff"] = df_val_preds.apply(lambda row: abs(row["val_preds"] - row["score"]), axis=1)
df_val_preds = df_val_preds.sort_values(by=["score_pred_diff"], ascending=False)
df_val_preds_diff = df_val_preds[df_val_preds.score_pred_diff > 0.2]
len(df_val_preds_diff)

5992

In [33]:
df_val_preds_diff[["anchor", "target", "context", "score", "val_preds", "score_pred_diff"]]

,anchor,target,context,score,val_preds,score_pred_diff
501,multiplexed data,multiplexed date,H01,0.00,1.026367,1.026367
4841,dissolving,dissolution,C22,0.00,0.979004,0.979004
6018,boom hydraulic cylinder,boom a hydraulic cylinder,E02,0.00,0.907715,0.907715
5469,sphygmomanometer,blood pressure,G09,1.00,0.132080,0.867920
6427,punch face,face punch,B05,0.00,0.838867,0.838867
...,...,...,...,...,...,...
3224,operating channel,channel of operation,H04,1.00,0.799805,0.200195
2827,synthetic training,virtual training,G06,0.50,0.700195,0.200195
4881,perform working operations,perform operations,B23,0.75,0.549805,0.200195
7249,define panel,solar energy,C02,0.00,0.200073,0.200073
